<a href="https://colab.research.google.com/github/Falakejaz786/customer-service/blob/main/Copy_of_customerservice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import requests

In [ ]:
GROQ_API_KEY = input("🔑 Enter your Groq API key (starts with gsk-...): ")

In [ ]:
user_id = input("👤 Enter user ID (e.g., user123): ")


In [ ]:
conversation_store = {}

In [ ]:
conversation_store[user_id] = [
    {
        "role": "system",
        "content": (
            "You are a helpful and polite customer support assistant for an e-commerce company. "
            "Always provide clear, accurate, and friendly responses. Ask for an order ID if the user mentions an order, "
            "help with product, return, refund, or delivery issues. Escalate to a human if you're unsure."
        )
    }
]


In [ ]:
def get_ai_response(history):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama3-8b-8192",  # or mixtral-8x7b-32768 if you want longer context
        "messages": history
    }

    response = requests.post(url, headers=headers, json=payload)

    try:
        data = response.json()
    except Exception:
        return f"❌ Invalid JSON response: {response.text}"

    if "choices" in data:
        return data["choices"][0]["message"]["content"]
    elif "error" in data:
        return f"❌ API Error: {data['error']['message']}"
    else:
        return f"❌ Unexpected response: {data}"

In [ ]:
input_box = widgets.Text(placeholder='Type your message here...')
output_area = widgets.Output()

def on_message_submit(change):
    user_message = input_box.value.strip()
    input_box.value = ""

    if not user_message:
        return

    # Add user message to conversation
    conversation_store[user_id].append({"role": "user", "content": user_message})

    # Display user message
    with output_area:
        print(f"You: {user_message}")

        # Get AI response
        ai_response = get_ai_response(conversation_store[user_id])
        print(f"AI: {ai_response}")

    # Add AI response to conversation history
    conversation_store[user_id].append({"role": "assistant", "content": ai_response})

input_box.on_submit(on_message_submit)

In [ ]:
display(input_box, output_area)